In [267]:
import numpy as np
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols
from scipy.linalg import block_diag
import scipy.optimize as opt
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [268]:
# Define constants
m1, m2, m3 = 1, 2, 0.5
L1, L2 = 0.5, 1.5
I1, I2 = 0.5, 1.5
F0 = 50
k = 1
g = 9.81
n = 1
phi = [2 * sp.pi * i / n for i in range(n)]


# Decide whether to use the realistically modelled forces (slow) or the simplified ones (faster)
useComplexForces = False

# Force parameters
P0 = 30000.0
volumeAv = 275.0
volumeChange = 225.0
scale = 1.396
tempStart = 100.0
tempSpike = 100.0 # Reduced temperature and temperature spike for better operating conditions
expFactor = -5.0
moleFactor = 6.0e-2
moleOffset = 3.12e-2
gasConstant = 8.314

# Friction parameters
lCrank = 0.08       # Length of crank
eta = 0.25     # Pa·s (motor oil) 
d = 0.030      # thickness of oil film (m)
D = 0.0757      # 75.7 mm
h = 0.04        # 40 mm
d1 = 0.00002     # 20 microns


# Initial conditions
dtheta1 = 20.0
theta_crank_0 = np.pi/3
phiNum = [float(phi[i].evalf()) for i in range(n)]
initial_angle = [theta_crank_0 + phiNum[i] for i in range(n)]



In [269]:
# Define t as symbol
t = sp.Symbol('t')

# Define generalized coordinates and their derivatives
x1, y1, theta1, x2, y2, theta2, x3, y3 = [[] for _ in range(8)]
for i in range(n):
    x1.append(dynamicsymbols(f'x1_{i}'))
    y1.append(dynamicsymbols(f'y1_{i}'))
    theta1.append(dynamicsymbols(f'theta1_{i}'))
    x2.append(dynamicsymbols(f'x2_{i}'))
    y2.append(dynamicsymbols(f'y2_{i}'))
    theta2.append(dynamicsymbols(f'theta2_{i}'))
    x3.append(dynamicsymbols(f'x3_{i}'))
    y3.append(dynamicsymbols(f'y3_{i}'))

# Create generalized coordinate vector
qMatrices = [sp.Matrix([x1[i], y1[i], theta1[i], x2[i], y2[i], theta2[i], x3[i], y3[i]]) for i in range(n)]
q = sp.Matrix.vstack(*qMatrices)
dq = q.diff(t)

# Center of mass positions
x_com_1 = [sp.Matrix([x1[i], y1[i]]) for i in range(n)]
x_com_2 = [sp.Matrix([x2[i], y2[i]]) for i in range(n)]
x_com_3 = [sp.Matrix([x3[i], y3[i]]) for i in range(n)]

# Rotation matrix
R = lambda theta: sp.Matrix([[sp.cos(theta), -sp.sin(theta)], [sp.sin(theta), sp.cos(theta)]])

# Mass matrix
massMatrices = [np.diag([m1, m1, I1, m2, m2, I2, m3, m3]) for _ in range(n)]
M = block_diag(*massMatrices)
W = np.linalg.inv(M)


In [270]:
# Declare forces
driveForce, fCrank, fPiston = [], [], []


def viscousFrictionCrank(N, lCrank, eta, L1, d):
    # Calculate viscous torque
    torque = (4 * np.pi**2 * eta * lCrank * L1**3 * N) / d
    return torque

def viscousFrictionPiston(v, eta, D, h, d1):
    A = np.pi * D * h  # side surface area of piston
    F = eta * A * v / d1
    return F


for i in range(n):
    # For use in local applications with phase shift as it starts at vertical, not 0
    thetaShifted = theta1[i] - sp.pi/2
    thetaLocal = sp.Mod(thetaShifted, 2 * sp.pi)

    # Initialise functions for temperature, force and volume
    temperature = (tempSpike * (1 - sp.exp(expFactor * (thetaLocal / scale) ** 2))) + tempStart
    moles = (moleFactor * (1 - sp.exp(expFactor * (thetaLocal / scale) ** 2))) + moleOffset
    volume = (volumeChange * -sp.cos(thetaShifted) + volumeAv) / 1e6

    # For exhaust stroke
    tempEnd = (tempSpike * (1 - sp.exp(expFactor * (3 * sp.pi / scale) ** 2))) + tempStart
    molesEnd = (moleFactor * (1 - sp.exp(expFactor * (3 * sp.pi / scale) ** 2))) + moleOffset
    volumeEnd = (volumeChange * -sp.cos(3 * sp.pi) + volumeAv) / 1e6
    pEnd = (molesEnd * gasConstant * tempEnd) / volumeEnd
    exhaustSlope = - pEnd / sp.pi

    # Pressure for each of the 4 strokes
    intakePressure = -P0 * sp.Heaviside(sp.sin(thetaShifted), 1) * sp.Heaviside(sp.sin(0.5 * thetaShifted), 1)
    compressionPressure = ((P0 * (volumeAv / 1e6) / volume) + P0) * sp.Heaviside(-sp.cos(0.5 * thetaShifted + sp.pi / 4), 1) * sp.Heaviside(-sp.sin(thetaShifted), 1)
    combustionPressure = ((moles * gasConstant * temperature) / volume) * sp.Heaviside(sp.sin(thetaShifted), 1) * sp.Heaviside(-sp.cos(0.5 * thetaShifted), 1)
    exhaustPressure = (pEnd + exhaustSlope * (thetaLocal - sp.pi)) * sp.Heaviside(-sp.sin(thetaShifted), 1) * sp.Heaviside(-sp.sin(0.5 * thetaShifted), 1)
    exhaustPressure = 0
    compressionPressure = 0
    intakePressure = 0

    # Combined pressure and force
    pressure = intakePressure + compressionPressure + combustionPressure + exhaustPressure
    driveForce.append(pressure * 4.5e-3)

    print(driveForce[i])

    fCrank.append(viscousFrictionPiston(theta1[i].diff(t), lCrank, eta, L1, d))
    fPiston.append(viscousFrictionCrank(y3[i].diff(t), eta, D, h, d1))

if useComplexForces == True:
    # Generalized forces
    forceMatrices = [sp.Matrix([0, -m1*g, -k*theta1[i].diff(t) - fCrank[i], 0, -m2*g, 0, 0, -m3*g - driveForce[i] - fPiston[i]]) for i in range(n)]
else:
    forceMatrices = [sp.Matrix([0, -m1*g, -k*theta1[i].diff(t), 0, -m2*g, 0, 0, -m3*g + F0 * sp.cos(theta1[i])]) for i in range(n)]

Q = sp.Matrix.vstack(*forceMatrices)


0.0045*(0.7582368 - 0.49884*exp(-2.56566038045665*(Mod(theta1_0(t) + 3*pi/2, 2*pi))**2))*(200.0 - 100.0*exp(-2.56566038045665*(Mod(theta1_0(t) + 3*pi/2, 2*pi))**2))*Heaviside(-cos(0.5*theta1_0(t) - 0.25*pi), 1)*Heaviside(-cos(theta1_0(t)), 1)/(0.000275 - 0.000225*sin(theta1_0(t)))


In [271]:
# Unit vectors
i_cap = sp.Matrix([1, 0])
j_cap = sp.Matrix([0, 1])

# Constraints
constraintMatrices = []
for i in range(n):
    constraint_1 = x_com_1[i] + R(theta1[i]) @ sp.Matrix([-L1/2, 0])
    C1 = constraint_1.dot(i_cap)
    C2 = constraint_1.dot(j_cap)

    constraint_2 = x_com_1[i] - x_com_2[i] + R(theta1[i]) @ sp.Matrix([L1/2, 0]) + R(theta2[i]) @ sp.Matrix([L2/2, 0])
    C3 = constraint_2.dot(i_cap)
    C4 = constraint_2.dot(j_cap)

    constraint_3 = x_com_2[i] + R(theta2[i]) @ sp.Matrix([L2/2, 0]) - x_com_3[i]
    C5 = constraint_3.dot(i_cap)
    C6 = constraint_3.dot(j_cap)

    constraint_4 = x_com_3[i][0]
    C7 = constraint_4

    if i == 0:
        constraintMatrices.append(sp.Matrix([C1, C2, C3, C4, C5, C6, C7]))
    else:
        constraint_5 = theta1[i - 1] - theta1[i] - phi[i]
        C8 = constraint_5
        constraintMatrices.append(sp.Matrix([C1, C2, C3, C4, C5, C6, C7, C8]))

# Full constraint matrix
C = sp.Matrix.vstack(*constraintMatrices)


In [272]:
J = C.jacobian(q)
dC = J @ dq
dJ = dC.jacobian(q)
JWJT = J @ W @ J.T
alpha = 9.99999943400706
beta = 0.1000108927910055
RHS = -dJ @ dq - J @ W @ Q - alpha * C - beta * dC

# Lambdify symbolic expressions
JWJT_fn = sp.lambdify((q, dq), JWJT, modules='numpy')
RHS_fn = sp.lambdify((q, dq), RHS, modules='numpy')
C_fn = sp.lambdify(q, C, modules='numpy')
J_fn = sp.lambdify((q, dq), J, modules='numpy')
dC_fn = sp.lambdify((q, dq), dC, modules='numpy')
dJ_fn = sp.lambdify((q, dq), dJ, modules='numpy')
Q_fn = sp.lambdify((q, dq), Q, modules='numpy')

In [273]:
def compute_initial_crank_position(L1, R_mat):
    """Compute the initial position of the crank pin."""
    vec = np.array([float(L1)/2, 0], dtype=float)  # Ensure L1 is float
    return R_mat @ vec

def compute_initial_rod2_position(L1, L2, R_mat, crank_pin):
    """Compute the initial position of the connecting rod's center."""
    L2 = float(L2)  # Convert L2 to float
    crank_pin = np.array(crank_pin, dtype=float)  # Ensure crank_pin is numeric
    y_offset = np.sqrt(max(0, L2**2 - crank_pin[0]**2))
    piston_pin = np.array([0, crank_pin[1] + y_offset], dtype=float)
    return (crank_pin + piston_pin) / 2

def compute_initial_piston_position(L1, L2, R_mat, crank_pin):
    """Compute the initial position of the piston."""
    L2 = float(L2)  # Convert L2 to float
    crank_pin = np.array(crank_pin, dtype=float)  # Ensure crank_pin is numeric
    dx = crank_pin[0]
    y_offset = np.sqrt(max(0, L2**2 - dx**2))
    return np.array([0.0, crank_pin[1] + y_offset], dtype=float)

def compute_rod2_angle(L1, R_mat, y_piston, crank_pin):
    """Compute the angle of the connecting rod."""
    y_piston = float(y_piston)  # Convert y_piston to float
    crank_pin = np.array(crank_pin, dtype=float)  # Ensure crank_pin is numeric
    piston_pos = np.array([0.0, y_piston], dtype=float)
    vec = piston_pos - crank_pin
    angle = np.arctan2(vec[1], vec[0])
    return angle

def generate_initial_guess(n, L1, L2, initial_angle):
    """Generate initial guess for positions and angles of crank, rod, and piston."""
    positions = []
    for i in range(n):
        angle = float(initial_angle[i])  # Ensure numerical value
        R_mat = R(angle)  # Create rotation matrix

        # Compute positions
        xy1 = compute_initial_crank_position(L1, R_mat)  # Crank pin position
        xy3 = compute_initial_piston_position(L1, L2, R_mat, xy1)  # Piston position
        xy2 = compute_initial_rod2_position(L1, L2, R_mat, xy1)  # Rod center position
        theta2 = compute_rod2_angle(L1, R_mat, xy3[1], xy1)  # Rod angle

        # Combine positions and angles
        if i == 0:
            pos1 = np.array([xy1[0], xy1[1]], dtype=float)  # Crank: x, y, theta
        else:
            pos1 = np.array([xy1[0], xy1[1], angle], dtype=float)  # Crank: x, y, theta
        pos2 = np.array([xy2[0], xy2[1], theta2], dtype=float)  # Rod: x, y, theta
        pos3 = np.array([xy3[0], xy3[1]], dtype=float)  # Piston: x, y

        # Flatten each position and extend
        positions.extend(pos1.tolist() + pos2.tolist() + pos3.tolist())

    return np.array(positions, dtype=float)


# Create an initial guess
p_guess = generate_initial_guess(n, L1, L2, initial_angle)
print(p_guess)

[0.125      0.21650635 0.0625     0.96389765 1.65422641 0.
 1.71128894]


In [274]:
def position_optimiser(p, n, theta_crank_0):
    x = np.zeros(8 * n)
    x[0:8] = [
        p[0],      # x1_0
        p[1],      # y1_0
        theta_crank_0, # theta1_0 fixed to theta_crank_0
        p[2],      # x2_0
        p[3],      # y2_0
        p[4],      # theta2_0
        p[5],      # x3_0
        p[6],      # y3_0
    ]
    offset = 7
    for i in range(1, n):
        start_x = 8 * i
        start_p = offset + (i - 1) * 8
        x[start_x:start_x + 8] = p[start_p:start_p + 8]
    return C_fn(*x).flatten()
    
pos_result = opt.root(position_optimiser, p_guess, args=(n, theta_crank_0), method='hybr')

if not pos_result.success:
    print("Position optimization failed:", pos_result.message)
else:
    print("Position optimization successful:", pos_result.message)

# Construct optimized positions
p = pos_result.x
x = np.zeros(8 * n)
x[0:8] = [
    p[0],
    p[1],
    theta_crank_0,  # Ensure theta1_0 is theta_crank_0
    p[2],
    p[3],
    p[4],
    p[5],
    p[6],
]
offset = 7
for i in range(1, n):
    start_x = 8 * i
    start_p = offset + (i - 1) * 8
    x[start_x:start_x + 8] = p[start_p:start_p + 8]


# Velocity optimization
def velocity_optimiser(b, n, dtheta1, x):
    dq = np.zeros(8 * n)
    dq[:8] = [
            b[0], b[1], dtheta1,  # dx1_0, dy1_0, dtheta1
            b[2], b[3], b[4],  # dx2_0, dy2_0, dtheta2_0
            b[5], b[6]  # dx3_0, dy3_0
    ]

    offset = 7
    for i in range(1, n):
        start_dq = 8 * i
        start_b = offset + (i - 1) * 8
        dq[start_dq:start_dq + 8] = b[start_b:start_b + 8]  

    return dC_fn(x, dq).flatten()

b_guess = np.zeros(8 * n - 1)
vel_result = opt.root(velocity_optimiser, b_guess, args=(n, dtheta1, x), method='hybr', options={'xtol': 1e-12, 'maxfev': 50000})

if not vel_result.success:
    print("Velocity optimization failed:", pos_result.message)
else:
    print("Velocity optimization successful:", vel_result.message)

# Construct full velocity vector
b = vel_result.x
dx = np.zeros(8 * n)
dx[:8] = [
    b[0], b[1], dtheta1,  # dx1_0, dy1_0, dtheta1
    b[2], b[3], b[4],  # dx2_0, dy2_0, dtheta2_0
    b[5], b[6]  # dx3_0, dy3_0
]

offset = 7
for i in range(1, n):
    start_dx = 8 * i
    start_b = offset + (i - 1) * 8
    dx[start_dx:start_dx + 8] = b[start_b:start_b + 8]  



# Verify constraints
C_val = C_fn(*x)
dC_val = dC_fn(x, dx)

print(f'Position constraint: {C_val}')
print(f'Velocity constraint: {dC_val}')

# Check constraints
C_val = np.array(C_val, dtype=np.float64)
dC_val = np.array(dC_val, dtype=np.float64)
assert np.allclose(C_val, 0, atol=1e-6), "Initial position constraint violated"
assert np.allclose(dC_val, 0, atol=1e-6), "Initial velocity constraint violated"

# Final initial condition
x0 = np.concatenate((x, dx))
print("Initial conditions (x0):", x0)

Position optimization successful: The solution converged.
Velocity optimization failed: The solution converged.
Position constraint: [[ 0.00000000e+00]
 [ 0.00000000e+00]
 [-5.55111512e-16]
 [ 4.05231404e-15]
 [-5.55111512e-16]
 [ 3.99680289e-15]
 [-1.20259179e-29]]
Velocity constraint: [[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-4.44089210e-16]
 [ 0.00000000e+00]
 [ 4.44089210e-16]
 [ 1.70617894e-33]]
Initial conditions (x0): [ 1.25000000e-01  2.16506351e-01  1.04719755e+00  1.25000000e-01
  1.17252267e+00  1.73824441e+00 -1.20259179e-29  1.91203265e+00
 -4.33012702e+00  2.50000000e+00  2.00000000e+01 -4.33012702e+00
  5.73192505e+00 -5.85540044e+00  1.70617894e-33  6.46385011e+00]


In [275]:
# Dynamics
def piston_engine(t, state):
    q, dq = np.split(state, 2)
    lam, _, _, _ = np.linalg.lstsq(JWJT_fn(q, dq), RHS_fn(q, dq), rcond=None)
    Qhat = J_fn(q, dq).T @ lam
    ddq = W @ (Q_fn(q, dq) + Qhat)
    ddq = ddq.flatten()
    return np.concatenate((dq, ddq))

# Solve dynamics
t_span = (0, 30)
t_eval = np.linspace(*t_span, 500)
sol = solve_ivp(piston_engine, t_span, x0, atol=1e-5, rtol=1e-5, method='Radau', t_eval=t_eval)


In [276]:
# Animation
class Box:
    def __init__(self, width, height, color='b'):
        self.width = width
        self.height = height
        self.color = color
        self.offset = -np.array([width/2, height/2])

    def first_draw(self, ax):
        corner = np.array([0, 0])
        self.patch = plt.Rectangle(corner, 0, 0, angle=0, 
                                   rotation_point='center', color=self.color, animated=True)
        ax.add_patch(self.patch)
        self.ax = ax
        return self.patch
    
    def set_data(self, x, y, theta):
        self.x = x
        self.y = y
        self.theta = theta

    def update(self, i):
        x, y, theta = self.x[i], self.y[i], self.theta[i]
        theta = np.rad2deg(theta)
        corner = np.array([x, y]) + self.offset
        self.patch.set_width(self.width)
        self.patch.set_height(self.height)
        self.patch.set_xy(corner)
        self.patch.set_angle(theta)
        return self.patch


In [277]:
# Animation setup
fig, ax = plt.subplots()
plt.close()

ax.set_ylim(-1, 3.0)  # Adjusted to show motion starting at np.pi/3
ax.set_xlim(-1.5, 1.5)
ax.set_aspect('equal')

# Create boxes for all systems
boxes = []
colors = ['b', 'r', 'g', 'm']
for i in range(n):
    x_i = sol.y[8 * i]
    y_i = sol.y[8 * i + 1]
    theta_i = sol.y[8 * i + 2]
    x2_i = sol.y[8 * i + 3]
    y2_i = sol.y[8 * i + 4]
    theta2_i = sol.y[8 * i + 5]
    x3_i = sol.y[8 * i + 6]
    y3_i = sol.y[8 * i + 7]
    theta3_i = np.full_like(x3_i, 0)
    
    box_crank = Box(L1, 0.01, colors[i % len(colors)])
    box_rod = Box(L2, 0.01, colors[i % len(colors)])
    box_piston = Box(0.1, 0.3, colors[i % len(colors)])
    
    box_crank.set_data(x_i, y_i, theta_i)
    box_rod.set_data(x2_i, y2_i, theta2_i)
    box_piston.set_data(x3_i, y3_i, theta3_i)
    
    boxes.extend([box_crank, box_rod, box_piston])

def init():
    ax.set_title("t=0.00 sec", fontsize=15)
    for box in boxes:
        box.first_draw(ax)
    return [box.patch for box in boxes]

def animate(i):
    ax.set_title(f"t={sol.t[i]:.2f} sec", fontsize=15)
    for box in boxes:
        box.update(i)
    return [box.patch for box in boxes]

dt = sol.t[1] - sol.t[0]
anim = FuncAnimation(fig, animate, frames=len(sol.t), init_func=init, blit=False, interval=1000*dt)
HTML(anim.to_html5_video())